In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

from dgp import DGP
from gformula import Gformula

scenario1 = {
    'K': 2,
    'P': 1,
    'C': None,
    'L': {
        'coefs': np.array(
            [0, 0, 0, 1, 0, -1]
        ),
        'sigma': np.array([1])
    },
    'A': {
        'coefs': np.array(
            [-3, 0, 1, 0, 0, 1]
        )
    },
    'D': None,
    'Y': {
        'coefs': np.array(
            [0, 0, 1, 0, -1, 0]
        ),
        'sigma': 1,
        'eof': False,
        'family': 'normal',
        'survival': False
    }
}

dgp = DGP(scenario1)
d = dgp.generate_data(N=10000, seed=1433235)

gform = Gformula(d, "id", "time")

gform.create_lags({'L1': 1, 'A': 1})

gform.outcome_model(
    formula="Y ~ A + L1", 
    family=sm.families.family.Gaussian()
)

gform.covariate_models(
    {
        'L1': {
            'formula': "L1 ~ lag1_L1 + lag1_A",
            'family': sm.families.family.Gaussian()
        },
        'A': {
            'formula': "A ~ L1 + lag1_A",
            'family': sm.families.family.Binomial()
        }
    }
)

gform.fit()

mc = gform.simulate( 
    interventions=[
    {
        'label': 'set A=1',
        'variables': 'A',
        'g': lambda h, time : 1},
    {
        'label': 'set A=0',
        'variables': 'A',
        'g': lambda h, time : 0},
    {
        'label': 'draw A from ~Bin(0.25)',
        'variables': 'A',
        'g': lambda h, time : np.random.binomial(1, 0.25, size=1000000)},
    {
        'label': 'set A=1 if L1>0.5',
        'variables': 'A',
        'g': lambda h, time : np.where(h['L1'] > 0.25, 1, 0)}
    ],
    lags=['A', 'L1'],
    samples=1000000,
    start=0,
    stop=1,
    seed=1433235
)

mc.summary()

# mc.bootstrap(reps=1000)

# mc.summary()


                  Intervention      Mean
0  natural course (parametric) -0.175241
1                      set A=1 -2.002385
2                      set A=0 -0.004794
3       draw A from ~Bin(0.25) -0.507042
4            set A=1 if L1>0.5 -0.704796


In [2]:
mc.sims[1]

,_uid,id,time,C,L1,A,D,Y,lag1_L1,lag1_A,_pY
0,0,1057,0,0.0,0.076521,1,0.0,-0.098795,0.000000,0.0,-0.886676
1,0,1057,1,0.0,-0.800777,1,0.0,-1.687149,0.076521,1.0,-1.768440
2,1,7315,0,0.0,-0.512656,1,0.0,-3.859610,0.000000,0.0,-1.478852
3,1,7315,1,0.0,-1.583673,1,0.0,-2.978642,-0.512656,1.0,-2.555321
4,2,8652,0,0.0,0.360058,1,0.0,-1.274679,0.000000,0.0,-0.601696
...,...,...,...,...,...,...,...,...,...,...,...
1999995,999997,8228,1,0.0,-2.160606,1,0.0,-3.340914,-0.086642,1.0,-3.135190
1999996,999998,3017,0,0.0,1.663114,1,0.0,3.457950,0.000000,0.0,0.707993
1999997,999998,3017,1,0.0,1.526330,1,0.0,-0.159539,1.663114,1.0,0.570513
1999998,999999,1326,0,0.0,2.197676,1,0.0,0.631356,0.000000,0.0,1.245277


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

from dgp import DGP
from gformula import Gformula

import os
import shutil
import socket
import datetime
import pickle
import torch
import matplotlib.pyplot as plt
from .utils import myconf, get_logger, loss_ISD, loss_KLD, loss_MPJPE
from .dvae.model import build_VAE, build_DKF, build_STORN, build_VRNN, build_SRNN, build_RVAE, build_DSAE



scenario1 = {
    'K': 2,
    'P': 1,
    'C': None,
    'L': {
        'coefs': np.array(
            [0, 0, 0, 1, 0, -1]
        ),
        'sigma': np.array([1])
    },
    'A': {
        'coefs': np.array(
            [-3, 0, 1, 0, 0, 1]
        )
    },
    'D': None,
    'Y': {
        'coefs': np.array(
            [0, 0, 1, 0, -1, 0]
        ),
        'sigma': 1,
        'eof': False,
        'family': 'normal',
        'survival': False
    }
}

dgp = DGP(scenario1)
d = dgp.generate_data(N=10000, seed=1433235)



In [ ]:
scenario2 = {
    'K': 2,
    'P': 1,
    'C': None,
    'L': {
        'coefs': np.array(
            [0, 0, 1, 0, -1, 0]
        ),
        'sigma': np.array([1])
    },
    'A': {
        'coefs': np.array(
            [-3, 0, 1, 0, 0, 1]
        )
    },
    'D': None,
    'Y': {
        'coefs': np.array(
            [-1, 0, 1, 0, -1, 0]
        ),
        'sigma': 1,
        'eof': False,
        'family': 'binomial',
        'survival': False
    }
}


dgp = DGP(scenario2)

dgp.generate_data(N = 100)

In [ ]:
scenario3 = {
    'K': 2,
    'P': 1,
    'C': None,
    'L': {
        'coefs': np.array(
            [0, 0, 1, 0, -1, 0]
        ),
        'sigma': np.array([1])
    },
    'A': {
        'coefs': np.array(
            [-3, 0, 1, 0, 0, 1]
        )
    },
    'D': None,
    'Y': {
        'coefs': np.array(
            [0, 0, 1, 0, -1, 0]
        ),
        'sigma': 1,
        'eof': True,
        'family': 'normal',
        'survival': False
    }
}


dgp = DGP(scenario3)

dgp.generate_data(N = 100)

In [ ]:
scenario1 = {   
    'K': 10,
    'P': 5,
    'C': {   
        'coefs': np.array([-4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])},
    'L': {   
        'coefs': np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0]]),
        'sigma': np.array([1., 1., 1., 1., 1.])}, 
    'A': {   
        'coefs': np.array([-3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])},
    'D': {   
        'coefs': np.array([-4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])},
    'Y': {   
        'coefs': np.array([-5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
        0,  0]),
        'eof': False,
        'family': 'binomial',
        'sigma': None,
        'survival': True}}